<a href="https://colab.research.google.com/github/SebastianBentert/example_code/blob/main/BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install python-constraint

In [50]:
import time
import numpy as np
from constraint import Problem
import itertools
from collections import defaultdict
import ipywidgets as widgets
from IPython.display import display

In [51]:
# Dictionaries to store base cases proven in various papers
matrizes_with_know_upper_bound = {}
matrizes_with_know_lower_bound = {}

In [52]:
def create_matrix(ones_indices):
  """
  Takes as input a list of 2-dimensional indices, e.g. [(0,0),(1,1)] and turns in into a numpy array with 1's in those indices and 0 everywhere else
  """
  rows = max([row for (row, column) in ones_indices])+1
  cols = max([column for (row, column) in ones_indices])+1
  a = np.zeros((rows, cols), dtype=int)
  for row, col in ones_indices:
    a[row, col] = 1
  return a

In [53]:
def generate_transformations(input_array):
  """
  Returns all possible rotations and reflections of the input matrix
  """
  transformations = []
  # 90-degree rotations
  for _ in range(4):
      input_array = np.rot90(input_array)
      transformations.append(input_array)
      horizontal_reflection = np.flip(input_array, axis=1)
      vertical_reflection = np.flip(input_array, axis=0)
      transformations.append(horizontal_reflection)
      transformations.append(vertical_reflection)


  return transformations

In [54]:
def add_entry_upper(ones_indices, upperbound):
  """
  Takes as input a list of 2-dimensional indices and corresponding upperbound, and adds the matrix (and it's rotations/reflections) into the upperbound dictionary
  """
  a = create_matrix(ones_indices)
  for transformation in generate_transformations(a):
    matrizes_with_know_upper_bound[np.array_str(transformation)] = {'arr': transformation, 'upperbound': upperbound}
  return

In [55]:
def add_entry_lower(ones_indices, lowerbound):
  """
  Takes as input a list of 2-dimensional indices and corresponding lowerbound, and adds the matrix (and it's rotations/reflections) into the lowerbound dictionary
  """
  a = create_matrix(ones_indices)
  for transformation in generate_transformations(a):
    matrizes_with_know_lower_bound[np.array_str(transformation)] = {'arr': transformation, 'lowerbound': lowerbound}
  return

In [56]:
def isLightWeight5(matrix):
  """
  Checks if a matrix is light and has weight 5 to apply Theorem 6.4 of "Generalized Davenport–Schinzel sequences and their 0–1 matrix counterparts"
  """
  return (matrix.shape[0] == 5 or matrix.shape[1] == 5) and np.sum(matrix) == 5

In [57]:
asym = {'n':1,'nα(n)':2,'nα(n)^2':3,'n2^α(n)':4,'nα(n)2^α(n)':5,'(n log n)/(log log n)':6,'n log n':7}
# Results not in asym are approximations by Davenport-Schinzel sequence 'n2^(α(n)^t/t!)' between 4 and 5 and approximations by Kővári–Sós–Turán theorem 'n^{2-1/min([n,m])}'

def isAsymptoticSmaller(str1,str2):
  """
  Checks if str2 is smaller than str1
  """
  if str1 == 'n^2':
    return True
  elif str1 in asym and str2 in asym:
    return asym[str2] < asym[str1]
  elif str1 in asym and str2.startswith('n^'):
    return False
  elif str1 in asym and str2.startswith('n2^(α(n)'):
    return 4 <= asym[str1]
  elif str2 in asym and str1.startswith('n^'):
    return True
  elif str2 in asym and str1.startswith('n2^(α(n)'):
    return asym[str2] <= 4
  elif str1.startswith('n^') and str2.startswith('n2^(α(n)'):
    return True
  elif str2.startswith('n^') and str1.startswith('n2^(α(n)'):
    return False
  elif str1.startswith('n2^(α(n)') and str2.startswith('n2^(α(n)'):
    return eval(str2[9:str2.find('/')]) < eval(str1[9:str1.find('/')])
  elif str1.startswith('n^') and str2.startswith('n^'):
    return eval(str2[2:]) < eval(str1[2:])

In [58]:
def is_permutation_matrix(matrix):
    """
    Checks if a matrix is an m-tuple permutation matrix
    """
    flag = False
    for i in range(matrix.shape[0]):
      if np.sum(matrix[i, :]) != 1:
        # Not light in rows
        flag = True
        break
    for j in range(matrix.shape[1]):
      if np.sum(matrix[:,j]) != 1:
        # Not light in columns
        if flag:
          return False
        break
    if not flag:
      matrix = matrix.transpose()
    for i in range(matrix.shape[0]):
      # Checks if all 1's in each row are consecutive
      indices = np.where(matrix[i,:] == 1)[0]
      if indices[0] + len(indices)-1 != indices[-1]:
        return False
    return True

In [59]:
def P_in_M(P,M):
  """
  Takes as input two 2-dimensional 0-1 numpy matrices.
  Returns True iff M contains P.
  """
  if P.shape[0] > M.shape[0] or P.shape[1] > M.shape[1]:
    return False

  # Extracting indices of 1-entries
  P = list(zip(*np.where(P == 1)))
  M = list(zip(*np.where(M == 1)))

  if len(P) > len(M):
    return False

  # Initialize the problem
  problem = Problem()

  # Add variables
  # The variables are the indices of the 1-entries in P
  # The domains are the indices of the 1-entries in M
  for index in P:
      problem.addVariable(index, M)

  # Sort variables based on x-coordinate
  sorted_P_by_x = sorted(P, key=lambda x: x[0])

  # Add relative order constraints for x-coordinate
  for i in range(len(sorted_P_by_x) - 1):
      x1, y1 = sorted_P_by_x[i]
      x2, y2 = sorted_P_by_x[i + 1]

      def constraint_x(val1, val2, x1=x1, x2=x2):
          if x1 == x2:
              return val1[0] == val2[0]
          elif x2 > x1:
              return val1[0] < val2[0]
          return True

      problem.addConstraint(constraint_x, [sorted_P_by_x[i], sorted_P_by_x[i + 1]])

  # Sort variables based on y-coordinate
  sorted_P_by_y = sorted(P, key=lambda x: x[1])

  # Add relative order constraints for y-coordinate
  for i in range(len(sorted_P_by_y) - 1):
      x1, y1 = sorted_P_by_y[i]
      x2, y2 = sorted_P_by_y[i + 1]

      def constraint_y(val1, val2, y1=y1, y2=y2):
          if y1 == y2:
              return val1[1] == val2[1]
          elif y2 > y1:
              return val1[1] < val2[1]
          return True

      problem.addConstraint(constraint_y, [sorted_P_by_y[i], sorted_P_by_y[i + 1]])

  # Find solutions
  solutions = problem.getSolutions()

  return len(solutions) > 0

In [60]:
# Davenport-Schinzel theory of matrices
# C2
add_entry_upper([(0,0),(0,1),(1,0),(1,2)],'n log n')
# C5 (proof in On 0–1 matrices and small excluded submatrices)
add_entry_upper([(0,0),(1,1),(2,0),(2,2)],'(n log n)/(log log n)')
# C6
add_entry_upper([(0,0),(0,2),(1,1),(1,3)],'nα(n)')

# Linear bound on extremal functions of some forbidden patterns in 0-1 matrices
# L2
add_entry_upper([(0,1),(0,2),(0,3),(1,0),(1,4),(2,2)],'n')

# Degrees of nonlinearity in forbidden 0–1 matrix problems
# H1
add_entry_upper([(0,1),(0,2),(1,3),(2,3),(3,0)],'n log n')

# Generalized Davenport–Schinzel sequences and their 0–1 matrix counterparts
# E3
add_entry_upper([(0,5),(1,1),(1,2),(2,3),(2,4),(2,6),(3,0)],'n')
# D1
add_entry_upper([(0,1),(0,2),(0,4),(1,0),(1,3)],'nα(n)^2')
# E1
add_entry_upper([(0,1),(0,2),(0,5),(0,6),(1,0),(1,3),(1,4),(1,7)],'n2^α(n)')
# D2
add_entry_upper([(0,3),(0,4),(1,0),(2,1),(2,2),(2,5)],'nα(n)')
# E2
add_entry_upper([(0,1),(1,3),(2,0),(2,2),(2,4)],'nα(n)^2')
# E5
add_entry_upper([(0,3),(0,4),(1,0),(1,7),(2,1),(2,2),(2,5),(2,6)],'nα(n)^2')

In [61]:
# Davenport-Schinzel theory of matrices
# C4
add_entry_lower([(0,0),(0,1),(1,0),(2,2)],'n log n')
# C5
add_entry_lower([(0,0),(1,1),(2,0),(2,2)],'(n log n)/(log log n)')
# C8
add_entry_lower([(0,0),(1,2),(2,1),(2,3)],'nα(n)')

# Degrees of nonlinearity in forbidden 0–1 matrix problems
# H1
add_entry_lower([(0,1),(0,2),(1,3),(2,3),(3,0)],'n log n')

# Generalized Davenport–Schinzel sequences and their 0–1 matrix counterparts
# E1
add_entry_lower([(0,1),(0,3),(1,0),(1,2),(1,4)],'n2^α(n)')

# E2
add_entry_lower([(0,1),(1,3),(2,0),(2,2),(2,4)],'nα(n)^2')


In [62]:
def find_upperbound(a, verbose=False):
  """
  Uses known reductions to look for the best possible asymptotic upperbound
  """
  # Delete empty rows/columns
  zero_rows = np.all(a == 0, axis=1)
  zero_cols = np.all(a == 0, axis=0)
  a = a[~zero_rows, :]
  a = a[:, ~zero_cols]

  if a.shape == (0, 0):
    if verbose:
      return float('-inf'), str(a), 'empty matrix'
    else:
      return float('-inf')
  if a.shape == (1, 1):
    if verbose:
      return '0', str(a), 'weight 1'
    else:
      return '0'

  # Initilize search
  stack = [a]
  result = 'n^2'
  respath = str(a)
  paths = [str(a)]
  reason = 'trivial'
  visited = set(str(a))

  # Search
  while stack:
    a = stack.pop()
    path = paths.pop()

    if is_permutation_matrix(a):
      # m-tuple permutation matrices have linear extremal functions
      if verbose:
        return ('n',path, 'permutation matrix')
      else:
        return 'n'

    if isAsymptoticSmaller(result,"n2^α(n)"):
      # Light matrices with weight 5 have an extremal function in O(n2^α(n))
      if isLightWeight5(a):
        result = "n2^α(n)"
        respath = path
        reason = 'Light with weight 5'

    n, m = a.shape
    weight = np.sum(a)

    if (n == 2 and weight == m) or (m == 2 and weight == n):
      # Finds the smallest matrix of form
      # 0 1 0 1 0 1 0
      # 1 0 1 0 1 0 1...
      # containing a
      if isAsymptoticSmaller(result,'nα(n)2^α(n)') or result.startswith('n2^(α(n)'):
        # Any Davenport-Schinzel sequences with length < 5 are contained in other results
        counter = 0
        if n == 2:
          last = a[0,0] == 1
          # True if last 1 was in first row
          for i in range(1,m):
            next = a[0,i] == 1
            if next == last:
              counter += 2
            else:
              counter += 1
            last = next
        elif m == 2:
          last = a[0,0] == 1
          # True if last 1 was in first column
          for i in range(1,n):
            next = a[i,0] == 1
            if next == last:
              counter += 2
            else:
              counter += 1
            last = next
        if counter == 5:
          if isAsymptoticSmaller(result,'nα(n)2^α(n)'):
            result = 'nα(n)2^α(n)'
            respath = path
            reason = 'Davenport–Schinzel sequence sequence of length 5'
        elif counter > 5:
          t = counter // 2 - 1
          if isAsymptoticSmaller(result,f'n2^(α(n)^{t}/{t}!)'):
            result = f'n2^(α(n)^{t}/{t}!)'
            respath = path
            reason = f'Davenport–Schinzel sequence sequence of length {counter}'


    # The full a x b matrix has an extremal function of n^2-(1/(min(a,b)))
    # and any a x b matrix is contained by the full a x b matrix
    if isAsymptoticSmaller(result,f'n^{2-1/min([n,m])}'):
      result = f'n^{2-1/min([n,m])}'
      respath = path
      reason = 'Kővári–Sós–Turán theorem'
    # Uses the reduction from "Davenport-Schinzel theory of matrices"
    if np.sum(a[0,:]) == 1:
      # If the matrix has a single 1 in the first row, we set it's neighbour in the 2nd row to 1 and delete the first row
      position = np.where(a[0, :] == 1)[0][0]
      temp = a[1:,:].copy()
      temp[0][position] = 1
      if str(temp) not in visited:
        visited.add(str(temp))
        stack.append(temp)
        paths.append(path + "->" + str(temp))
    if np.sum(a[-1,:]) == 1:
      # Last row
      position = np.where(a[-1, :] == 1)[0][0]
      temp = a[:-1,:].copy()
      temp[-1][position] = 1
      if str(temp) not in visited:
        visited.add(str(temp))
        stack.append(temp)
        paths.append(path + "->" + str(temp))
    if np.sum(a[:,0]) == 1:
      # First column
      position = np.where(a[:, 0] == 1)[0][0]
      temp = a[:,1:].copy()
      temp[position][0] = 1
      if str(temp) not in visited:
        visited.add(str(temp))
        stack.append(temp)
        paths.append(path + "->" + str(temp))
    if np.sum(a[:,-1]) == 1:
      # Last column
      position = np.where(a[:, -1] == 1)[0][0]
      temp = a[:,:-1].copy()
      temp[position][-1] = 1
      if str(temp) not in visited:
        visited.add(str(temp))
        stack.append(temp)
        paths.append(path + "->" + str(temp))

    # Uses the reduction from Theorem 2.3 of "On linear forbidden submatrices"
    if np.sum(a[0,:]) == 2:
      ind1, ind2 = np.where(a[0] == 1)[0]
      if ind1 == ind2-1 and ind1 > 0 and ind2 < m-1 and np.sum(a[:,ind1]) == 1 and np.sum(a[:,ind2]) == 1:
        # If there is only two 1's in the first row and those 1's are the only only 1's in their respective column
        # We can set the diagonale outwards neighbours to 1 and delete the first row (and the now empty columns)
        temp = np.hstack([a[1:, :ind1], a[1:, ind2+1:]])
        temp[0][ind1-1] = temp[0][ind2-1] = 1
        if str(temp) not in visited:
          visited.add(str(temp))
          stack.append(temp)
          paths.append(path + "->" + str(temp))
    if np.sum(a[-1,:]) == 2:
      ind1, ind2 = np.where(a[-1] == 1)[0]
      if ind1 == ind2-1 and ind1 > 0 and ind2 < m-1 and np.sum(a[:,ind1]) == 1 and np.sum(a[:,ind2]) == 1:
        # Last row
        temp = np.hstack([a[:-1, :ind1], a[:-1, ind2+1:]])
        temp[-1][ind1-1] = temp[-1][ind2-1] = 1
        if str(temp) not in visited:
          visited.add(str(temp))
          stack.append(temp)
          paths.append(path + "->" + str(temp))
    if np.sum(a[:,0]) == 2:
      ind1, ind2 = np.where(a[:,0] == 1)[0]
      if ind1 == ind2-1 and ind1 > 0 and ind2 < n-1 and np.sum(a[ind1,:]) == 1 and np.sum(a[ind2,:]) == 1:
        # First column
        temp = np.vstack([a[:ind1, 1:], a[ind2+1:, 1:]])
        temp[ind1-1][0] = temp[ind2-1][0] = 1
        if str(temp) not in visited:
          visited.add(str(temp))
          stack.append(temp)
          paths.append(path + "->" + str(temp))
    if np.sum(a[:,-1]) == 2:
      ind1, ind2 = np.where(a[:,-1] == 1)[0]
      if ind1 == ind2-1 and ind1 > 0 and ind2 < n-1 and np.sum(a[ind1,:]) == 1 and np.sum(a[ind2,:]) == 1:
        # Last column
        temp = np.vstack([a[:ind1, :-1], a[ind2+1:, :-1]])
        temp[ind1-1][-1] = temp[ind2-1][-1] = 1
        if str(temp) not in visited:
          visited.add(str(temp))
          stack.append(temp)
          paths.append(path + "->" + str(temp))

    # Uses the reduction from Theorem 2.2 of "On linear forbidden submatrices" by Keszegh
    for arr,ncols,nrows in [(a,m,n),(np.rot90(a),n,m)]:
      # We check for empty blocks in top right and bottom left so that only a single row and column are inbetween
      # Afterwards we rotate to check for top left and bottom right
      right = ncols-1
      left = 0
      # The ith value in upperrightblocks gives the minimal indice so that a[:i+1,upperrightblocks[i]:] is empty
      # The ith value in lowerleftblocks gives the maximal indice so that a[-(i+1):,:lowerleftblocks[i]+1] is empty
      upperrightblocks = []
      lowerleftblocks = []

      while arr[0,right] == 0:
        # On first row we find the last 1
        right -=1
      for i in range(nrows):
        # On each consecutive row, we walk right until we passed all 1's
        counter = np.sum(arr[i,right:])
        while counter:
          if arr[i,right] == 1:
            counter -= 1
          right += 1
        upperrightblocks.append(right)
      while arr[-1,left] == 0:
        left +=1
      for i in range(nrows-1,-1,-1):
        counter = np.sum(arr[i,:left+1])
        while counter:
          if arr[i,left] == 1:
            counter -= 1
          left -= 1
        lowerleftblocks.append(left)
      for i in range(nrows-2):
        if upperrightblocks[i] <= lowerleftblocks[-(i+3)]+2:
          # upperrightblocks[i] corresponds to a row 2 above lowerleftblocks[-(i+3)],
          # If there is at most 1 row between them we can split the matrix on the row/column inbetween
          lowerright = arr[i+1:,upperrightblocks[i]-1:].copy()
          upperleft = arr[:i+2,:upperrightblocks[i]].copy()
          # We have to set the entry in which the two matrices intersect to 1
          lowerright[0,0] = 1
          upperleft[-1,-1] = 1
          if min(lowerright.shape) > 1 and min(upperleft.shape) > 1:
            # If we only effectively cut off a single row or column, we will get the same result with the "Davenport-Schinzel theory of matrices" reduction
            res1, path1, reason1 = find_upperbound(lowerright, verbose=True)
            res2, path2, reason2 = find_upperbound(upperleft, verbose=True)
            if isAsymptoticSmaller(res1,res2):
              # First submatrix dominates the 2nd
              if isAsymptoticSmaller(result,res1):
                result = res1
                respath = f'{path}->({path2}),{path1}'
                reason = reason1
            else:
              if isAsymptoticSmaller(result,res2):
                result = res2
                respath = f'{path}->({path1}),{path2}'
                reason = reason2

    # Checks if a is contained in one of the known matrices
    for matrix in matrizes_with_know_upper_bound:
      if isAsymptoticSmaller(result,matrizes_with_know_upper_bound[matrix]['upperbound']):
        if P_in_M(a,matrizes_with_know_upper_bound[matrix]['arr']):
          result = matrizes_with_know_upper_bound[matrix]['upperbound']
          respath = path
          reason = f'contained by {matrix}'

  if verbose:
    return result, respath, reason
  else:
    return result

In [63]:
def find_lowerbound(a,verbose=False):
  """
  Uses known reductions to look for the best possible asymptotic lowerbound
  """
  # Delete empty rows/columns
  zero_rows = np.all(a == 0, axis=1)
  zero_cols = np.all(a == 0, axis=0)
  a = a[~zero_rows, :]
  a = a[:, ~zero_cols]

  if a.shape == (0, 0):
    if verbose:
      return float('-inf'), str(a), 'empty matrix'
    else:
      return float('-inf')
  if a.shape == (1, 1):
    if verbose:
      return '0', str(a), 'weight 1'
    else:
      return '0'

  # Initilize search
  stack = [a]
  result = 'n'
  respath = str(a)
  paths = [str(a)]
  reason = 'trivial'
  visited = set(str(a))

  # Search
  while stack:
    a = stack.pop()
    path = paths.pop()
    n, m = a.shape
    weight = np.sum(a)

    # The full a x b matrix has an extremal function of n^2-(1/(min(a,b)))
    lsq = 2
    while P_in_M(np.ones((lsq,lsq)),a):
      if not isAsymptoticSmaller(result,f'n^{2-1/lsq}'):
        result = f'n^{2-1/lsq}'
        respath = path
        reason = 'Kővári–Sós–Turán theorem'
      lsq +=1

    # Uses the reduction from "Davenport-Schinzel theory of matrices"
    if np.sum(a[0,:]) != 1:
      # If we have multiple 1's in the first row, we can move each one outwards seperately
      positions = np.where(a[0, :] == 1)[0]
      for position in positions:
        temp = np.zeros(m, dtype=int)
        temp[position] = 1
        temp = np.vstack((temp, a.copy()))
        temp[1, position] = 0
        if str(temp) not in visited:
          visited.add(str(temp))
          stack.append(temp)
          paths.append(path + "->" + str(temp))
    if np.sum(a[-1,:]) != 1:
      # Last row
      positions = np.where(a[-1, :] == 1)[0]
      for position in positions:
        temp = np.zeros(m, dtype=int)
        temp[position] = 1
        temp = np.vstack((a.copy(), temp))
        temp[-2,position] = 0
        if str(temp) not in visited:
          visited.add(str(temp))
          stack.append(temp)
          paths.append(path + "->" + str(temp))
    if np.sum(a[:,0]) != 1:
      # First column
      positions = np.where(a[:, 0] == 1)[0]
      for position in positions:
        temp = np.zeros(n, dtype=int)
        temp[position] = 1
        temp = np.hstack((temp.reshape(-1, 1), a.copy()))
        temp[position, 1] = 0
        if str(temp) not in visited:
          visited.add(str(temp))
          stack.append(temp)
          paths.append(path + "->" + str(temp))
    if np.sum(a[:,-1]) != 1:
      # Last column
      positions = np.where(a[:, -1] == 1)[0]
      for position in positions:
        temp = np.zeros(n, dtype=int)
        temp[position] = 1
        temp = np.hstack((a.copy(), temp.reshape(-1, 1)))
        temp[position, -2] = 0
        if str(temp) not in visited:
          visited.add(str(temp))
          stack.append(temp)
          paths.append(path + "->" + str(temp))

    # Checks if the matrix contains one of the known results
    for matrix in matrizes_with_know_lower_bound:
      if not isAsymptoticSmaller(result,matrizes_with_know_lower_bound[matrix]['lowerbound']):
        if P_in_M(matrizes_with_know_lower_bound[matrix]['arr'],a):
          result = matrizes_with_know_lower_bound[matrix]['lowerbound']
          respath = path
          reason = f'contains {matrix}'
  if verbose:
    return result, respath, reason
  else:
    return result

In [64]:
def find_bound(a,verbose=False):
  if verbose:
    return f"lowerbound: {find_lowerbound(a,verbose=True)} \nupperbound: {find_upperbound(a,verbose=True)}"
  if not verbose:
    return f"lowerbound: {find_lowerbound(a)} \nupperbound: {find_upperbound(a)}"

In [ ]:
class InteractiveMatrix:
    def __init__(self):
        self.rows_slider = widgets.IntSlider(min=1, max=10, value=3, description='Rows')
        self.cols_slider = widgets.IntSlider(min=1, max=10, value=3, description='Columns')
        self.create_button = widgets.Button(description='Create Matrix')
        self.confirm_button = widgets.Button(description='Confirm Matrix')
        self.matrix_output = widgets.Output()
        self.create_button.on_click(self.on_create_button_clicked)
        self.confirm_button.on_click(self.on_confirm_button_clicked)

    def display(self):
        display(widgets.VBox([self.rows_slider, self.cols_slider, self.create_button, self.confirm_button, self.matrix_output]))

    def on_create_button_clicked(self, b):
        self.matrix = np.zeros((self.rows_slider.value, self.cols_slider.value), dtype=int)
        self.draw_matrix()

    def draw_matrix(self):
        buttons = [[widgets.Button(description='', layout=widgets.Layout(width='30px', height='30px')) for _ in range(self.cols_slider.value)] for _ in range(self.rows_slider.value)]
        for i in range(self.rows_slider.value):
            for j in range(self.cols_slider.value):
                buttons[i][j].style.button_color = 'lightgray'  # Color for 0's
                buttons[i][j].on_click(self.create_on_click_handler(i, j))

        grid = widgets.GridBox(children=[button for row in buttons for button in row],
                               layout=widgets.Layout(grid_template_columns=("repeat(%d, 30px)" % self.cols_slider.value)))
        with self.matrix_output:
            self.matrix_output.clear_output(wait=True)
            display(grid)

    def create_on_click_handler(self, row, col):
        def on_click(b):
            self.matrix[row][col] = 1 - self.matrix[row][col]
            b.style.button_color = 'black' if self.matrix[row][col] == 1 else 'lightgray'
        return on_click

    def on_confirm_button_clicked(self, b):
        with self.matrix_output:
            print("Final Matrix:")
            print(self.matrix)
            print(find_bound(self.matrix,verbose=True))

interactive_matrix = InteractiveMatrix()
interactive_matrix.display()

In [66]:
def generate_matrices_with_weight_x(x):
    """
    Generates an iterator of all 0-1 matrices with weight x
    """
    # All possible positions in a x by x array
    all_positions = [(i, j) for i in range(x) for j in range(x)]

    # Generate all combinations of x positions
    for positions in itertools.combinations(all_positions, x):
        # Create an array filled with zeros
        array = np.zeros((x, x), dtype=int)

        # Place 1s in the specified positions
        for pos in positions:
            array[pos] = 1

        yield array

In [67]:
def generate_matrices_of_size_x(x):
    """
    Generates an iterator of all x by x 0-1 matrices
    """
    # Total number of positions in a 4x4 array
    total_positions = x * x

    # Iterate over all possible combinations of 0s and 1s
    for binary_string in itertools.product([0, 1], repeat=total_positions):
        # Convert the binary string to a 4x4 array
        array = np.array(binary_string).reshape(x, x)

        yield array

In [ ]:
# Analyses matrices based on weight
resultupper = defaultdict(int)
resultlower = defaultdict(int)
for i in range(6):
  print('matrices of weight', i)
  resultupper = defaultdict(int)
  resultlower = defaultdict(int)
  start_time = time.time()
  yes = 0
  no = 0
  visited = set()
  matrix_iterator = generate_matrices_with_weight_x(i)
  for i, arr in enumerate(matrix_iterator):
    # Filter duplicates
    zero_rows = np.all(arr == 0, axis=1)
    zero_cols = np.all(arr == 0, axis=0)
    arr = arr[~zero_rows, :]
    arr = arr[:, ~zero_cols]
    for transformation in generate_transformations(arr):
      if str(transformation) in visited:
        break
    else:
      visited.add(str(arr))
      resupper = find_upperbound(arr)
      reslower = find_lowerbound(arr)
      resultupper[resupper] += 1
      resultlower[reslower] += 1
      if resupper == reslower:
        yes += 1
      else:
        no += 1
  print('total: ',len(visited))
  print('found tight bound for: ', yes)
  print('distribution of upper bounds: ',resultupper)
  print('distribution of lower bounds: ',resultlower)
  # End timer
  end_time = time.time()

  # Calculate elapsed time
  elapsed_time = end_time - start_time
  print("Elapsed time: ", elapsed_time,'\n')

In [ ]:
# Analyses matrices based on size
resultupper = defaultdict(int)
resultlower = defaultdict(int)
for i in range(4):
  print(f'matrices of size {i}x{i}')
  resultupper = defaultdict(int)
  resultlower = defaultdict(int)
  start_time = time.time()
  yes = 0
  no = 0
  visited = set()
  matrix_iterator = generate_matrices_of_size_x(i)
  for i, arr in enumerate(matrix_iterator):
    # Filter duplicates
    zero_rows = np.all(arr == 0, axis=1)
    zero_cols = np.all(arr == 0, axis=0)
    arr = arr[~zero_rows, :]
    arr = arr[:, ~zero_cols]
    for transformation in generate_transformations(arr):
      if str(transformation) in visited:
        break
    else:
      visited.add(str(arr))
      resupper = find_upperbound(arr)
      reslower = find_lowerbound(arr)
      resultupper[resupper] += 1
      resultlower[reslower] += 1
      if resupper == reslower:
        yes += 1
      else:
        no += 1
  print('total: ',len(visited))
  print('found tight bound for: ', yes)
  print('distribution of upper bounds: ',resultupper)
  print('distribution of lower bounds: ',resultlower)
  # End timer
  end_time = time.time()

  # Calculate elapsed time
  elapsed_time = end_time - start_time
  print("Elapsed time: ", elapsed_time,'\n')